In [1]:
import numpy as np
import pandas as pd
import os
import re
import string
import urllib.parse
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import time
import socket
import whois
import tldextract
from tld import get_tld
from dask import delayed
from datetime import datetime

In [2]:
df=pd.read_csv(r"D:\spam_url\Otrim_azam\O_trim_prod_data_ham\unique_domain\datasets_spits\set_8.csv")

In [3]:
df.head(2)

,url
0,https://everydayparisian.com/where-to-eat-and-...
1,https://newsindiastate.blogspot.com/2023/02/on...


In [4]:
df.shape

(498, 1)

In [5]:
# urldata=df.drop(columns=['Unnamed: 0'], axis = 0)
# urldata.head(2)

In [6]:
urldata=df
urldata.head(2)

,url
0,https://everydayparisian.com/where-to-eat-and-...
1,https://newsindiastate.blogspot.com/2023/02/on...


In [7]:
urldata.shape

(498, 1)

# Lexical Features

In [8]:
#1
def calculate_underscore_count(url):
    underscore_count = url.count("_")
    return underscore_count
#2

def calculate_path_length(url):
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path
    path_length = len(path)
    return path_length

#3
def calculate_dot_count_host(url):
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.hostname
    dot_count = str(host).count(".")
    return dot_count

#4
def calculate_ampersand_count(url):
    ampersand_count = url.count("&")
    return ampersand_count

#5
def count_questionmark(url):
    que_count = url.count("?")
    return que_count

#6
def calculate_special_char_count(url):
    special_char_count = len(re.findall(r'[^A-Za-z0-9./]', url))
    return special_char_count   

#7
def count_punctuation(url):
    count = 0
    for char in url:
        if char in string.punctuation:
            count += 1
    return count

#8
def check_tld_in_path(url):
    tld_match = re.search("\.(\w+)/", url)
    if tld_match:
        tld = tld_match.group(1)
        path = url.split(tld)[0].split("/")[-1]
        return 1 if tld in path else 0
    else:
        return 0

#9
def count_digits_in_host(url):
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.hostname
    return sum(c.isdigit() for c in str(host))   

#10
def check_legitimate_url(url):
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.hostname
    domain = str(host).split("//")[-1]
    if "www." in domain or ".com" in domain:
        return 1
    else:
        return 0   

#11
def calculate_dot_count(url):
    dots_count = url.count(".")
    return dots_count


#12
def count_words_in_host(url):
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.hostname
    return len(str(host).split("."))

#13
def has_hyphen_in_path(url):
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path
    return 1 if "-" in path else 0

#14
def calculate_equals_count(url):
    equal_count = url.count("=")
    return equal_count

#15
def calculate_redirected(url):
    redirected_count = url.count("//")
    return redirected_count

#16
def url_without_www(url):
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname
    return 1 if not str(hostname).startswith("www.") else 0

#17
def has_hyphen_in_host(url):
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname
    return 1 if "-" in str(hostname) else 0

#18
def check_spam_url(url):
    
    if re.search("\?.*=.*", url):
        return 1
    else:
        return 0
#19
def count_subdomains(url):
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.hostname
    domain = str(host).split("//")[-1]
    subdomains = str(domain).split(".")
    return len(subdomains) - 1    

#20
def dots_in_path(url):
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path
    return path.count(".")

#21
def https_protocol(url):
    if url.startswith("https"):
        return 1
    else:
        return 0

#22
def count_repeated_characters(url):
    count = 0
    for i in range(len(url) - 1):
        if url[i] == url[i + 1]:
            count += 1
    return count

#23

def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)|'  # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
        '[0-9a-fA-F]{7}', url)  # Ipv6
    if match:
        return 1
    else:
        return 0


#24
def calculate_at_count(url):
    dots_count = url.count("@")
    return dots_count

#25
def calculate_hyphen_count(url):
    dots_count = url.count("-")
    return dots_count



#26
def port_number_present(url):
    try:
        parsed_url = urllib.parse.urlparse(url)
        if str(parsed_url.port):
            return 1
        else:
            return 0
    except:
        return -1

#27
def domain_name_length(url):
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname
    return len(str(hostname))

#28
def url_length(url):
    return len(url)

#29
def avg_word_length(url):
    words = re.findall(r'\w+', url)
    if len(words) == 0:
        return 0
    else:
        avg_length = sum(len(word) for word in words) / len(words)
        return round(float(avg_length),5)

#30
def shortest_word(url):
    words = re.findall(r'\b\w+\b', url)
    if len(words) == 0:
        return 0
    else:
        return min(len(word) for word in words)

#31
def longest_word_in_host_name(url):
    parsed_url = urllib.parse.urlparse(url)
    host_name = parsed_url.hostname
    host_name = str(host_name)
    if host_name is None:
        return 0
    longest_word_length = 0
    words = host_name.split('.')
    for word in words:
        if len(word) > longest_word_length:
            longest_word_length = len(word)
    return longest_word_length


#32
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

#33
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

#34
def vowel_consonant_ratio(url):
    vowels = "aeiouAEIOU"
    vowels_count = 0
    consonants_count = 0
    for char in url:
        if char.isalpha():
            if char in vowels:
                vowels_count += 1
            else:
                consonants_count += 1
    if consonants_count == 0:
        return 0
    else:
        cnt = vowels_count / consonants_count
        return round(float(cnt),5) if consonants_count != 0 else 0

#35
def digit_letter_ratio(url):
    digits = 0
    letters = 0
    for char in url:
        if char.isdigit():
            digits += 1
        elif char.isalpha():
            letters += 1
    return round(float(digits/letters),5) if letters != 0 else 0

#36
def word_length_diff(url):
    words = url.split(".")[0].split("/")[-1].split("-")
    word_lengths = [len(word) for word in words]
    return max(word_lengths) - min(word_lengths)

#37
def word_length_std(url):
    hostname = urllib.parse.urlparse(url).hostname
    hostname = str(hostname)
    words = hostname.split('.')
    word_lengths = [len(word) for word in words]
    return round(float(np.std(word_lengths)), 5)

#38
def tilde_symbol(url):
    if "~" in url:
        return 1
    else:
        return 0

#39
def num_numeric_chars(url):
    return sum(c.isdigit() for c in url)

#40
def calculate_per_count(url):
    dots_count = url.count("%")
    return dots_count

#41
def calculate_hash_count(url):
    dots_count = url.count("#")
    return dots_count

#42
def protocol_count(url):
    http_count = url.count('http')
    https_count = url.count('https')
    http_count = http_count - https_count #correcting the miscount of https as http
    return (http_count + https_count)

#43
def count_subdirectories(url):
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path
    return len(path.split('/')) - 1

#44
def percent_20_in_path(url):
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path
    return 1 if '%20' in path else 0

#45
def has_single_char_dirs(url):
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path
    dirs = path.split("/")
    single_char_dirs = [dir for dir in dirs if len(dir) == 1]
    return 1 if single_char_dirs else 0

#46
def count_zeros(url):
    return url.count('0')

#47
def calculate_ratio(url):
    upper_count = sum(c.isupper() for c in url)
    lower_count = sum(c.islower() for c in url)
    if lower_count == 0:
        return 0
    else:
        return round(float(upper_count / lower_count),5) 

#48
def count_or(url):
    return url.count('|')

#49
def count_star(url):
    return url.count('*')

#50
def count_colon(url):
    return url.count(':')

#51
def count_dollar(base_url):
     return base_url.count('$')    

#52
def count_semicolon(url):
     return url.count(';')   

#53
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return -1
    else:
        return 1
    

#54
def existenceOfSensitiveWords(url):
    phishingWords = ["secure","account","login","signin","confirm","submit", "webscr"]
    phishing = False
    for i in range(len(phishingWords)):
        if(phishingWords[i] in url):
            phishing = True
    if(phishing == True):
        return 1
    else:
        return 0
        

# #55
# def is_blacklisted(url):
#     blacklist_file = 'blacklists.txt'
#     with open(blacklist_file, 'r') as f:
#         blacklist = [re.escape(line.strip()) for line in f]

#     tokens = re.findall(r'\w+', url)
#     for token in tokens:
#         if token.lower() in blacklist:
#             return 1

#     return 0


#56
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

#57
def get_tld_length(url):
    try:
        tld = get_tld(url, fail_silently=True)
        return len(tld) if tld else -1
    except:
        return -1


#58
# def is_suspicious(url):
#     try:
#         with open('suspicious_tlds.txt', 'r') as f:
#             suspicious_tlds = set(line.strip() for line in f)

#         ext = tldextract.extract(url)
#         tld = ext.suffix

#         return 1 if tld in suspicious_tlds else 0
#     except:
#         return -1
    
#59
def punycode(url):
    if url.startswith("http://xn--") or url.startswith("http://xn--"):
        return 1
    else:
        return 0

In [9]:
columns=['calculate_underscore_count','calculate_path_length','calculate_dot_count_host','calculate_ampersand_count',
        'count_questionmark','calculate_special_char_count','count_punctuation','check_tld_in_path',
       'count_digits_in_host','check_legitimate_url','calculate_dot_count','count_words_in_host',
        'has_hyphen_in_path','calculate_equals_count','calculate_redirected','url_without_www','has_hyphen_in_host','check_spam_url',
       'count_subdomains','dots_in_path','https_protocol','count_repeated_characters','having_ip_address',
        'calculate_at_count','calculate_hyphen_count','port_number_present','domain_name_length',
        'url_length','avg_word_length','shortest_word','longest_word_in_host_name','digit_count','letter_count',
        'vowel_consonant_ratio','digit_letter_ratio','word_length_diff','word_length_std','tilde_symbol','num_numeric_chars',
        'calculate_per_count','calculate_hash_count','protocol_count','count_subdirectories','percent_20_in_path',
        'has_single_char_dirs','count_zeros','calculate_ratio','count_or','count_star','count_colon','count_dollar',
        'count_semicolon','shortening_service','existenceOfSensitiveWords',
        'fd_length','get_tld_length','punycode']

final_df1 = pd.DataFrame(columns=columns)
start_time = time.time()
print("Start time: ", start_time)
# Loop over URLs in input DataFrame and apply the final function to each URL
final_df1 = pd.DataFrame([delayed([delayed(globals()[columns[i]])(url) for i in range(len(columns))]).compute() for url in urldata['url']],columns=columns)
end_time = time.time()
difference = end_time - start_time
# Convert the difference from seconds to minutes
difference_in_minutes = difference / 60

print("End time: ", end_time)
print("Difference (in seconds): ", difference)
print("Difference (in minutes): ", difference_in_minutes)

Start time:  1690547576.0157168
End time:  1690547578.313919
Difference (in seconds):  2.2982022762298584
Difference (in minutes):  0.03830337127049764


In [10]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
final_df1.head()

,calculate_underscore_count,calculate_path_length,calculate_dot_count_host,calculate_ampersand_count,count_questionmark,calculate_special_char_count,count_punctuation,check_tld_in_path,count_digits_in_host,check_legitimate_url,calculate_dot_count,count_words_in_host,has_hyphen_in_path,calculate_equals_count,calculate_redirected,url_without_www,has_hyphen_in_host,check_spam_url,count_subdomains,dots_in_path,https_protocol,count_repeated_characters,having_ip_address,calculate_at_count,calculate_hyphen_count,port_number_present,domain_name_length,url_length,avg_word_length,shortest_word,longest_word_in_host_name,digit_count,letter_count,vowel_consonant_ratio,digit_letter_ratio,word_length_diff,word_length_std,tilde_symbol,num_numeric_chars,calculate_per_count,calculate_hash_count,protocol_count,count_subdirectories,percent_20_in_path,has_single_char_dirs,count_zeros,calculate_ratio,count_or,count_star,count_colon,count_dollar,count_semicolon,shortening_service,existenceOfSensitiveWords,fd_length,get_tld_length,punycode
0,0,33,1,0,0,7,12,0,0,1,1,2,1,0,1,1,0,0,1,0,1,2,0,0,6,1,20,61,4.90000,2,16,0,49,0.58065,0.00000,0,6.50000,0,0,0,0,1,2,0,0,0,0.00000,0,0,1,0,0,1,0,31,3,0
1,0,36,2,0,0,3,11,0,0,1,3,3,1,0,1,1,0,0,2,1,1,4,0,0,2,1,27,71,6.00000,2,14,6,54,0.45946,0.11111,0,4.49691,0,6,0,0,1,3,0,0,2,0.00000,0,0,1,0,0,-1,0,4,12,0
2,0,27,2,0,0,1,8,0,0,1,3,3,0,0,1,0,0,0,2,1,1,5,0,0,0,1,24,59,7.28571,3,16,0,51,0.37838,0.00000,0,6.12826,0,0,0,0,1,2,0,0,0,0.08511,0,0,1,0,0,1,0,9,3,0
3,0,13,2,1,1,5,11,0,0,1,3,3,0,2,1,0,0,1,2,1,1,4,0,0,0,1,22,61,5.00000,1,14,1,49,0.25641,0.02041,0,5.18545,0,1,0,0,1,1,0,0,0,0.00000,0,0,1,0,0,1,0,12,3,0
4,0,10,2,0,1,3,9,0,0,0,3,3,0,1,1,1,0,1,2,1,0,4,0,0,0,1,24,62,6.62500,3,14,0,53,0.55882,0.00000,0,4.71405,0,0,0,0,1,1,0,0,0,0.00000,0,0,1,0,0,1,0,9,4,0


In [11]:
final_df1.shape

(498, 57)

# Non - Lexical

In [12]:
def pct_ext_resource_urls(response, hostname, soup):
    try: 
        # Send a GET request to the URL
        response = requests.get(url)

        # Parse the HTML content of the response using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all the external resource URLs in the HTML source code
        ext_urls = [tag['src'] for tag in soup.find_all(src=True) if 'http' in tag['src']]

        # Calculate the percentage of external resource URLs in the HTML source code
        total_urls = len(soup.find_all(src=True))
        pct_ext_urls = round((len(ext_urls) / total_urls) * 100, 5)

        return pct_ext_urls
    except:
        return -1
    
def insecure_forms(response, hostname, soup):
    try:
        html_content = response.text
        forms = soup.find_all('form')
        for form in forms:
            if form.get('action').startswith('http://'):
                return 1
        return 0
    except:
        return -1


def relative_form_action(response,hostname,soup):
    try:
      forms = soup.find_all('form')
      for form in forms:
          action = form.get('action')
          if not action.startswith(('http://', 'https://')):
              return 1
      return 0
    except:
        return -1

def ExtFormAction(response,hostname,soup):
    try:
        forms = soup.find_all("form")
        ext_form = False
        for form in forms:
            action = form.get("action")
            if action:
                parsed_action = requests.utils.urlparse(action)
                action_hostname = parsed_action.hostname
                if action_hostname != hostname:
                    ext_form = True
                    break
        if ext_form:
            return 1
        else:
            return 0
    except:
        return -1



def is_normal_form_action(form_action_url):
    # a predefined list of known good URL structures
    normal_form_action_url_starts = [
        "https://",
        "http://",
        "mailto:"
    ]
    
    # check if the form action URL starts with any of the known good URL structures
    for normal_form_action_url_start in normal_form_action_url_starts:
        if form_action_url.startswith(normal_form_action_url_start):
            return True
    
    # if none of the known good URL structures match, return False
    return False
def AbnormalFormAction(response,hostname,soup):
    try: 
        forms = soup.find_all("form")
        # keep track of the number of abnormal form actions
        num_abnormal_form_actions = 0
        # for each form element, extract the URL specified in the action attribute
        for form in forms:
            form_action_url = form.get("action")
            # analyze the extracted URL to determine if it is abnormal or not
            if not is_normal_form_action(form_action_url):
                num_abnormal_form_actions += 1
        # return the result
        if num_abnormal_form_actions > 0:
            return 0
        else:
            return 1
    except:
        return -1

def frequent_domain_name_mismatch(response,hostname,soup):
    try:
        urls = [link.get('href') for link in soup.find_all('a')] + [url]
        hostnames = [requests.utils.urlparse(u).hostname for u in urls if u is not None]
        most_frequent_hostname = max(set(hostnames), key=hostnames.count)
        return 1 if most_frequent_hostname != requests.utils.urlparse(url).hostname else 0
    except:
        return -1

def pop_up_window(response,hostname,soup):
    try:
        scripts = soup.find_all('script')
        for script in scripts:
            if 'window.open' in script.text:
                return 1
        return 0
    except:
        return -1


def submit_info_to_email(response,hostname,soup):
    try:
        forms = soup.find_all('form')
        for form in forms:
            action = form.get('action')
            if action and 'mailto:' in action:
                return 1
        return 0
    except:
        return -1

def iframe_or_frame(response,hostname,soup):
    try:

        # Find all iframe and frame elements in the HTML content
        iframes = soup.find_all('iframe')
        frames = soup.find_all('frame')

        # If any iframe or frame elements are found, return 1
        if len(iframes) > 0 or len(frames) > 0:
            return 1

        # If no iframe or frame elements are found, return 0
        return 0
    except:
        return -1

def missing_title(response,hostname,soup):
    try:

        # Find the title tag in the HTML content
        title = soup.find('title')

        # If the title tag is found, return 0
        if title:
            return 0

        # If the title tag is not found, return 1
        return 1
    except:
        return -1

def src_eval_cnt(response,hostname,soup):
    try: 
        script_tags = soup.find_all("script")
        eval_cnt = 0
        for tag in script_tags:
            eval_cnt += tag.text.count("eval(")
        return eval_cnt
    except:
        return -1

def src_escape_cnt(response,hostname,soup):
    try:
        script_tags = soup.find_all("script")
        escape_cnt = 0
        for tag in script_tags:
            escape_cnt += tag.text.count(" escape(")
        return escape_cnt
    except:
        return -1

def src_exec_cnt(response,hostname,soup):
    try:
        html_content = response.text
        # Count the number of times the "exec" function is used in the HTML source code
        return html_content.count("exec(")
    except:
        return -1

def src_search_cnt(response,hostname,soup):
    try:
        html_content = response.text
        # Use regular expressions to search for instances of the search() function in the HTML content
        search_cnt = len(re.findall('search\(', html_content))
        return search_cnt
    except:
        return -1

def images_only_in_form(response,hostname,soup):
    try:
        forms = soup.find_all('form')
        # Loop through each form element and check if it only contains "img" elements
        for form in forms:
            if all(child.name == 'img' for child in form.children):
                return 1

        # If no form only contains "img" elements, return 0
        return 0
    except:
        return -1


def PctNullSelfRedirectHyperlinks(response,hostname,soup):
    try:
    
        # Find all the hyperlinks on the web page
        all_links = soup.find_all('a')

        # Count the number of hyperlinks that redirect to a null or blank page
        null_links = 0
        for link in all_links:
            if link.get('href') == '' or link.get('href')=="#" or link.get('href') is None:
                null_links += 1

        # Calculate the percentage of null links
        pct_null_links = null_links / len(all_links) * 100

        return round(float(pct_null_links),5)
    except:
        return -1

def links_pointing_to_page(response,hostname,soup):
    try:
        links = [link.get("href") for link in soup.find_all("a")]
        return len(links)
    except:
        return -1


def request_url(response, hostname, soup):
    try:
        external_objects = soup.find_all(['img', 'video', 'audio'])
        for obj in external_objects:
            src = obj.get('src')
            if requests.utils.urlparse(src).netloc:
                domain = requests.utils.urlparse(src).netloc
                url_domain = requests.utils.urlparse(response.url).netloc
                if domain != url_domain:
                    return 1
            else:
                src = obj.get('data-src')
                domain = requests.utils.urlparse(src).netloc
                url_domain = requests.utils.urlparse(response.url).netloc
                if domain != url_domain:
                    return 1
        return 0
    except:
        return -1

def linksmeta(response,hostname,soup):
    try:
        # Checking if the response is successful
        if response.status_code == 200:
            # Finding all the links in <Meta>, <Script>, and <Link> tags
            meta_links = [meta.get('content') for meta in soup.find_all('meta') if meta.get('content')]
            script_links = [script.get('src') for script in soup.find_all('script') if script.get('src')]
            link_links = [link.get('href') for link in soup.find_all('link') if link.get('href')]

            # Merging all the links into a single list
            all_links = meta_links + script_links + link_links

            # Initializing a count of links from the same domain
            same_domain_links_count = 0

            # Iterating over the links
            for link in all_links:
                # Checking if the link is from the same domain as the URL
                if link.startswith(url) or link.startswith(url.split('//')[-1].split('/')[0]):
                    same_domain_links_count += 1

            # Calculating the percentage of links from the same domain
            same_domain_links_percentage = same_domain_links_count / len(all_links) * 100

            # Return the result
            return round(float(same_domain_links_percentage),5)
        else:
            # If the response was not successful, return None
            return -1
    except:
        return -1

In [13]:
columns = ['pct_ext_resource_urls', 'insecure_forms', 'relative_form_action', 'ExtFormAction', 'AbnormalFormAction', 
           'frequent_domain_name_mismatch',
     'pop_up_window', 'submit_info_to_email', 'iframe_or_frame', 'missing_title', 
          'src_eval_cnt', 'src_escape_cnt', 'src_exec_cnt', 'src_search_cnt', 'images_only_in_form',  'PctNullSelfRedirectHyperlinks', 'links_pointing_to_page',
          'request_url', 'linksmeta']

lowercase_alphabets=list(string.ascii_lowercase)
alphabets=list(string.ascii_letters)[0:19]
def calc(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s):
    return a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s
def final_1(response,  hostname, soup):
    for z in range(len(alphabets)):
        globals()[alphabets[z]] = delayed(globals()[columns[z]])(response,  hostname, soup)
    return delayed(calc)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s).compute()

start_time = time.time()
print("Start time: ", start_time)
# Loop over URLs in input DataFrame and apply the final function to each URL
rows = []
for url in urldata['url']:
    print('url', url)
    aa = False
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}, verify = True, timeout=10)
        hostname = response.url.split("//")[-1].split("/")[0]
        soup = BeautifulSoup(response.text, 'html.parser')
    except:
        aa= True
    
#         print('url', url)
    if(aa):
        l1 = (-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1)
     
    else:
        l1 = final_1(response,  hostname, soup)
    # Call the final function on the URL
    rows.append(l1)
    
df = pd.DataFrame(rows,columns=columns)    
end_time = time.time()
difference = end_time - start_time
# Convert the difference from seconds to minutes
difference_in_minutes = difference / 60

print("End time: ", end_time)
print("Difference (in seconds): ", difference)
print("Difference (in minutes): ", difference_in_minutes)

Start time:  1690547578.4325616
url https://everydayparisian.com/where-to-eat-and-drink-in-paris/
url https://newsindiastate.blogspot.com/2023/02/onpassive-full-revives.html
url https://www.playindialottery.com/QuickLink/ResultChart.aspx
url https://www.adprofitcycler.com/splashpg.php?pid=2&r=surelinks
url http://opal.thebusinesswiz.info/index.php?referid=millionmiler
url http://betterdayads.com/index.php?referid=millionmiler
url http://pro.supertextmarketing.com/index.php?referid=millionmiler
url https://proorno.com/1001052407
url https://db643j9ermj6h6824q-gj13l6d.hop.clickbank.net
url https://www.safetydetective.com/article/best-chromebook-antivirus-apps/?cq_src=google_ads&cq_cmp=18713825928&cq_term=&cq_plac=&cq_net=g&cq_plt=gp&gclid=CjwKCAiAuaKfBhBtEiwAht6H75ubx5wB3xaYIx6ADbkeEXt6Vs0ImWEo-qO8uF-inKTKt7VZqOwqSRoCZBoQAvD_BwE
url http://rtr.real-time-traffic.net/index.php?referid=migrantduck
url https://intofinityservices.com/register.aspx?id=pavanjain513@gmail.com
url https://ecosyst

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


url https://app-cdn.minepi.com/mobile-app-ui/wallet/balance
url https://www.imagetotext.info/
url https://freetools.textmagic.com/text-to-speech
url https://navia.my.salesforce.com/setup/forcecomHomepage.apexp?setupid=ForceCom&retURL=%2Fhome%2Fhome.jsp%3Fsource%3Dlex
url https://extraincomewithpravin.blogspot.com/2017/04/here-you-will-find-multiple-sources-of.html
url http://kzncoin.app/r?f=adminvunguyen
url https://www.lowermyrx.com/card/?group=3125
url https://www.theprimalatom.com/reading-head-start/
url https://www.swagbucks.com/profile/PrimalAtom
url https://www.dreamsreallycometrue.com/optin1673585140102
url https://www.grassrootsnw.com/about-grassroots-nw/clients/parenting-now-formerly-birth-to-three-eugene-oregon/
url https://wordwall.net/he/resource/30579096/%D7%A6%D7%99%D7%A8%D7%95%D7%A4%D7%99%D7%9D-%D7%90%D7%95%D7%AA%D7%99%D7%95%D7%AA-%D7%9E%D7%93%D7%91%D7%A8%D7%95%D7%AA-%D7%97%D7%95%D7%91%D7%A8%D7%AA-%D7%A9%D7%A0%D7%99%D7%99%D7%94
url https://cultmeasly.buzz/
url https://dr

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


url https://dist.bzzworld.com/#/dashboard/SystemIndex
url https://shihtzu.exchange/dashboard
url https://greg-stukes-vigorous-income.aweb.page/p/474e4715-d3b5-4f7c-8718-2b3fd477ff0a
url https://docs.digitalocean.com/products/droplets/
url https://www.digitalocean.com/products/droplets
url https://www.mtdc.co/en/
url https://www.appcreator24.com/app2337554-uchj4x
url https://www.lolsurveys.com/referrer/fortimewall
url https://www.amarujala.com/atul-maheshwari-chhatravritti-2019
url https://naasongs.co/sir-telugu-songs-2022.html
url https://timebucks.com/publishers/index.php?pg=dashboard
url https://economictimes.indiatimes.com/defaultinterstitial.cms
url https://news.google.com/home?hl=en-IN&gl=IN&ceid=IN:en
url https://www.holidify.com/collections/tourist-places-in-india
url https://www.saregama.com/regional/telugu_21/song/top
url https://www.thehansindia.com/news/cities/hyderabad/todays-top-5-hyderabad-news-updates-783424?infinitescroll=1
url https://www.toptenreviews.com/best-cookboo

url http://w.existentranch.cn/flipkart-v22/tb.php?gp=dy1677574996303
url https://h5.vidilook.org/#/pages/login/login?code=B8R5NE
url https://rezkyaffiliate.blogspot.com/2023/01/bismillah-assalamualaikum-terima-kasih.html
url https://routine.com/markdouglass
url https://www.justballgloves.com/?rfsn=7168843.03fe88
url https://erc.bottomlinesavings.com/allen-mendez/
url https://www.justpaddles.com/?rfsn=7168850.c7e143
url https://moneyfunhealthtravelandmore.weebly.com/#/
url https://www.tumblr.com/thunder-bro
url https://amelianonresidentsettlor.my.webex.com/amelianonresidentsettlor.my/j.php?MTID=md7e3f92b4fba4b4b0c5576f359369f67
url https://iwebatool.net/0128x.php?user=rshadonn
url https://shoplivegood.com/bignews
url https://dashboard.mailerlite.com/forms/346474/81390216859878485/share
url https://325f4bxvmxq9uqjgmi03zjclvy.hop.clickbank.net/?cbpage=Splittest
url https://www.manishsedai.com
url https://social.zoho.in/social/testmail/115574000000022005/Home.do?isAccessRequested=true#repo

url https://ncclimited.com/governance-philosophy.html
url https://www.highrevenuegate.com/d4isaxt64?key=3f74bcdb70f3e01cda870bdd33f2fff0
url https://marplive.com/registrace/4b20af5189178a9b
url https://www.billdesk.com/pgidsk/pgijsp/sbicard/SBI_card.jsp
url https://www.ups.com/track?loc=en_US&requester=ST/trackdetails
url https://dtdt.page.link/Gjbv
url https://www.jotform.com/app/230645376798168
url https://www.freeimageconverter.com/resize/online/resizedphotos.aspx
url https://trafficize.app/app/FusionXperience
url https://evermoreacademydnn.wixsite.com/evermoreacademy
url https://peitlernico.wixsite.com/martin-treuhand
url https://api.transferwise.com/v1/notification-flow/messages/6016f5ab-4a71-4313-a8e3-c0dd2beaa850/deliveries/64614e44-a711-4e3b-8ec4-3f4a094ba761/linkClicks?name=invite&link=aHR0cHM6Ly93aXNlLmNvbS9pbnZpdGUjLw&hash=5ef8458f43109c93a33e9eeadb17bffd
url https://zubearalam.blogspot.com/2023/03/make-money-online-everybody.html
url http://offeringpsychiatric.cn/f6baYWp0Sg

In [14]:
df

,pct_ext_resource_urls,insecure_forms,relative_form_action,ExtFormAction,AbnormalFormAction,frequent_domain_name_mismatch,pop_up_window,submit_info_to_email,iframe_or_frame,missing_title,src_eval_cnt,src_escape_cnt,src_exec_cnt,src_search_cnt,images_only_in_form,PctNullSelfRedirectHyperlinks,links_pointing_to_page,request_url,linksmeta
0,92.68293,0,0,1,1,0,0,0,1,0,0,0,0,0,0,2.27273,264,1,4.68750
1,71.42857,-1,-1,0,-1,1,0,0,1,0,1,0,0,0,0,19.23077,104,1,7.89474
2,0.00000,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1.00000,0,1,0.00000
3,-1.00000,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.00000,-1,-1,-1.00000
4,28.57143,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0.00000,31,1,0.00000
5,37.50000,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0.00000,32,1,0.00000
6,28.57143,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0.00000,25,1,0.00000
7,3.73832,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0.95238,105,1,0.00000
8,0.00000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.00000,3,1,0.00000
9,100.00000,-1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,16.22642,265,1,0.00000


In [15]:
df2 = pd.concat([urldata,final_df1, df], axis = 1)

In [16]:
df2.head()

,url,calculate_underscore_count,calculate_path_length,calculate_dot_count_host,calculate_ampersand_count,count_questionmark,calculate_special_char_count,count_punctuation,check_tld_in_path,count_digits_in_host,check_legitimate_url,calculate_dot_count,count_words_in_host,has_hyphen_in_path,calculate_equals_count,calculate_redirected,url_without_www,has_hyphen_in_host,check_spam_url,count_subdomains,dots_in_path,https_protocol,count_repeated_characters,having_ip_address,calculate_at_count,calculate_hyphen_count,port_number_present,domain_name_length,url_length,avg_word_length,shortest_word,longest_word_in_host_name,digit_count,letter_count,vowel_consonant_ratio,digit_letter_ratio,word_length_diff,word_length_std,tilde_symbol,num_numeric_chars,calculate_per_count,calculate_hash_count,protocol_count,count_subdirectories,percent_20_in_path,has_single_char_dirs,count_zeros,calculate_ratio,count_or,count_star,count_colon,count_dollar,count_semicolon,shortening_service,existenceOfSensitiveWords,fd_length,get_tld_length,punycode,pct_ext_resource_urls,insecure_forms,relative_form_action,ExtFormAction,AbnormalFormAction,frequent_domain_name_mismatch,pop_up_window,submit_info_to_email,iframe_or_frame,missing_title,src_eval_cnt,src_escape_cnt,src_exec_cnt,src_search_cnt,images_only_in_form,PctNullSelfRedirectHyperlinks,links_pointing_to_page,request_url,linksmeta
0,https://everydayparisian.com/where-to-eat-and-...,0,33,1,0,0,7,12,0,0,1,1,2,1,0,1,1,0,0,1,0,1,2,0,0,6,1,20,61,4.90000,2,16,0,49,0.58065,0.00000,0,6.50000,0,0,0,0,1,2,0,0,0,0.00000,0,0,1,0,0,1,0,31,3,0,92.68293,0,0,1,1,0,0,0,1,0,0,0,0,0,0,2.27273,264,1,4.68750
1,https://newsindiastate.blogspot.com/2023/02/on...,0,36,2,0,0,3,11,0,0,1,3,3,1,0,1,1,0,0,2,1,1,4,0,0,2,1,27,71,6.00000,2,14,6,54,0.45946,0.11111,0,4.49691,0,6,0,0,1,3,0,0,2,0.00000,0,0,1,0,0,-1,0,4,12,0,71.42857,-1,-1,0,-1,1,0,0,1,0,1,0,0,0,0,19.23077,104,1,7.89474
2,https://www.playindialottery.com/QuickLink/Res...,0,27,2,0,0,1,8,0,0,1,3,3,0,0,1,0,0,0,2,1,1,5,0,0,0,1,24,59,7.28571,3,16,0,51,0.37838,0.00000,0,6.12826,0,0,0,0,1,2,0,0,0,0.08511,0,0,1,0,0,1,0,9,3,0,0.00000,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1.00000,0,1,0.00000
3,https://www.adprofitcycler.com/splashpg.php?pi...,0,13,2,1,1,5,11,0,0,1,3,3,0,2,1,0,0,1,2,1,1,4,0,0,0,1,22,61,5.00000,1,14,1,49,0.25641,0.02041,0,5.18545,0,1,0,0,1,1,0,0,0,0.00000,0,0,1,0,0,1,0,12,3,0,-1.00000,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.00000,-1,-1,-1.00000
4,http://opal.thebusinesswiz.info/index.php?refe...,0,10,2,0,1,3,9,0,0,0,3,3,0,1,1,1,0,1,2,1,0,4,0,0,0,1,24,62,6.62500,3,14,0,53,0.55882,0.00000,0,4.71405,0,0,0,0,1,1,0,0,0,0.00000,0,0,1,0,0,1,0,9,4,0,28.57143,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0.00000,31,1,0.00000


In [17]:
df2.to_csv(r"D:\spam_url\Otrim_azam\O_trim_prod_data_ham\unique_domain\datasets_spits\features\set_8.csv")